# 02. Multi-Head Attention 직접 구현

## 학습 목표

-   Multi-Head Attention의 동작 원리 이해
-   Head 분리와 병합 과정 구현
-   왜 여러 head가 필요한지 이해

## 관련 면접 질문

-   Multi-Head Attention이 왜 필요한가요?
-   Head 수를 늘리면 무엇이 좋아지나요?
-   d_model, num_heads, d_k의 관계는?

## 핵심 수식

$$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, ..., \text{head}_h)W^O$$ $$\text{where } \text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$$


---

## Pre-Quiz

### Q1. Single Head 대신 Multi-Head를 쓰는 이유는?

```
당신의 답:
```

### Q2. d_model=512, num_heads=8이면 d_k는?

```
당신의 답:
```

### Q3. 각 head가 학습하는 것은?

```
당신의 답:
```


---

## Setup


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
from typing import Optional

plt.rcParams["font.family"] = "AppleGothic"
plt.rcParams["axes.unicode_minus"] = False

torch.manual_seed(42)

In [ ]:
# 01에서 구현한 scaled_dot_product_attention (제공)
def scaled_dot_product_attention(
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    mask: Optional[torch.Tensor] = None,
) -> tuple[torch.Tensor, torch.Tensor]:
    d_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask, float("-inf"))
    weights = F.softmax(scores, dim=-1)
    output = torch.bmm(weights, value)
    return output, weights

---

## 1. Multi-Head Attention 구현

### 핵심 아이디어

-   하나의 attention 대신 **h개의 head**로 병렬 처리
-   각 head는 **다른 관계 패턴**을 학습
    -   head 1: 주어-동사 관계
    -   head 2: 형용사-명사 관계
    -   head 3: 시간 관계 등...

### Shape 변환 과정

```
Input: (batch, seq, d_model)
    ↓ Q, K, V projection
(batch, seq, d_model)
    ↓ reshape to heads
(batch, seq, num_heads, d_k)
    ↓ transpose
(batch, num_heads, seq, d_k)
    ↓ attention (각 head 독립 계산)
(batch, num_heads, seq, d_k)
    ↓ transpose + reshape (concat)
(batch, seq, d_model)
    ↓ output projection
Output: (batch, seq, d_model)
```


In [ ]:
# =================================================
# TODO: Multi-Head Attention 구현
# =================================================
#
# $$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, ..., \text{head}_h)W^O$$
# $$\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$$
#
# 핵심: d_model을 num_heads로 나눠서 병렬 처리
#       d_k = d_model // num_heads
#
# Shape 변환:
#   (B, T, d_model) → (B, h, T, d_k) → attention → (B, T, d_model)
#
# =================================================


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, num_heads: int, dropout: float = 0.0):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.dropout = nn.Dropout(dropout)

        # 4개의 Linear layers 정의
        self.WQ = nn.Linear(self.d_model, self.d_model)
        self.WK = nn.Linear(self.d_model, self.d_model)
        self.WV = nn.Linear(self.d_model, self.d_model)

        self.WO = nn.Linear(self.d_model, self.d_model)

    def forward(
        self,
        query: torch.Tensor,
        key: torch.Tensor,
        value: torch.Tensor,
        mask: Optional[torch.Tensor] = None,
    ) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Args:
            query: (B, T_q, d_model)
            key: (B, T_k, d_model)
            value: (B, T_k, d_model)

        Returns:
            output: (B, T_q, d_model)
            weights: (B, h, T_q, T_k)
        """
        batch_size = query.size(0)
        seq_q = query.size(1)  # query는 질문에 해당하므로 size가 다를 수 있음
        seq_k = seq_v = key.size(1)  # key와 value는 항상 같은 size임

        # 위 수식과 shape 변환을 구현하세요
        Q = self.WQ(query)
        K = self.WQ(key)
        V = self.WQ(value)

        Q = Q.view(batch_size, seq_q, self.num_heads, self.d_k)
        K = K.view(batch_size, seq_k, self.num_heads, self.d_k)
        V = V.view(batch_size, seq_v, self.num_heads, self.d_k)

        Q = Q.transpose(batch_size, self.num_heads, seq_q, self.d_k)
        K = K.transpose(batch_size, self.num_heads, seq_k, self.d_k)
        V = V.transpose(batch_size, self.num_heads, seq_v, self.d_k)

        Q = Q.contiguous().view(batch_size, seq_q, self.num_heads, d_k)
        K = K.contiguous().view(batch_size, seq_k, self.num_heads, d_k)
        V = V.contiguous().view(batch_size, seq_v, self.num_heads, d_k)

        if mask:
            pass
            

        output, weight = scaled_dot_product_attention(Q, K, V, mask)

        output = output.


        return output, weight

In [ ]:
# Self-Attention wrapper (편의용)
class MultiHeadSelfAttention(MultiHeadAttention):
    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None):
        return super().forward(x, x, x, mask)

In [ ]:
# ============ 테스트 1: Shape 확인 ============
def test_shape():
    batch, seq, d_model, num_heads = 2, 10, 64, 8

    mha = MultiHeadSelfAttention(d_model, num_heads)
    x = torch.randn(batch, seq, d_model)

    output, weights = mha(x)

    assert output.shape == (batch, seq, d_model), f"Output shape 오류: {output.shape}"
    assert weights.shape == (batch, num_heads, seq, seq), f"Weights shape 오류: {weights.shape}"
    print("✅ Shape 테스트 통과!")
    print(f"   Input: {x.shape}")
    print(f"   Output: {output.shape}")
    print(f"   Attention weights: {weights.shape}")


test_shape()

In [ ]:
# ============ 테스트 2: Cross Attention ============
def test_cross_attention():
    batch, seq_q, seq_k, d_model, num_heads = 2, 8, 12, 64, 8

    mha = MultiHeadAttention(d_model, num_heads)
    q = torch.randn(batch, seq_q, d_model)
    k = torch.randn(batch, seq_k, d_model)
    v = torch.randn(batch, seq_k, d_model)

    output, weights = mha(q, k, v)

    assert output.shape == (batch, seq_q, d_model)
    assert weights.shape == (batch, num_heads, seq_q, seq_k)
    print("✅ Cross Attention 테스트 통과!")
    print(f"   Q: {q.shape}, K: {k.shape}, V: {v.shape}")
    print(f"   Output: {output.shape}")


test_cross_attention()

In [ ]:
# ============ 테스트 3: Attention weights 합 검증 ============
def test_attention_sum():
    batch, seq, d_model, num_heads = 2, 6, 64, 8

    mha = MultiHeadSelfAttention(d_model, num_heads)
    x = torch.randn(batch, seq, d_model)

    _, weights = mha(x)

    # 각 head의 각 query에 대해 weights 합이 1
    row_sums = weights.sum(dim=-1)  # (batch, num_heads, seq)
    expected = torch.ones_like(row_sums)

    assert torch.allclose(row_sums, expected, atol=1e-5), "Weights 합이 1이 아님!"
    print("✅ Attention weights 합 검증 통과!")


test_attention_sum()

---

## 2. Head별 Attention 시각화

각 head가 다른 패턴을 학습하는지 확인합니다.


In [ ]:
def visualize_multihead_attention(weights, tokens=None, figsize=(16, 4)):
    """각 head의 attention weights 시각화"""
    if weights.dim() == 4:
        weights = weights[0]  # 첫 번째 배치

    num_heads = weights.shape[0]
    seq_len = weights.shape[1]

    if tokens is None:
        tokens = [f"pos{i}" for i in range(seq_len)]

    fig, axes = plt.subplots(1, num_heads, figsize=figsize)

    for h in range(num_heads):
        ax = axes[h] if num_heads > 1 else axes
        im = ax.imshow(weights[h].detach().numpy(), cmap="Blues", vmin=0, vmax=0.5)
        ax.set_title(f"Head {h+1}")
        ax.set_xticks(range(seq_len))
        ax.set_yticks(range(seq_len))
        if h == 0:
            ax.set_xticklabels(tokens, rotation=45, ha="right", fontsize=8)
            ax.set_yticklabels(tokens, fontsize=8)
        else:
            ax.set_xticklabels([])
            ax.set_yticklabels([])

    plt.suptitle("Multi-Head Attention Weights (각 head가 다른 패턴 학습)", y=1.02)
    plt.tight_layout()
    plt.show()


# 시각화 테스트
batch, seq, d_model, num_heads = 1, 6, 64, 4
tokens = ["I", "love", "natural", "language", "processing", "."]

mha = MultiHeadSelfAttention(d_model, num_heads)
x = torch.randn(batch, seq, d_model)
_, weights = mha(x)

visualize_multihead_attention(weights, tokens)

---

## 3. PyTorch 공식 구현과 비교


In [ ]:
def test_vs_pytorch():
    batch, seq, d_model, num_heads = 2, 8, 64, 8

    # 동일한 가중치로 초기화하여 비교
    torch.manual_seed(42)
    our_mha = MultiHeadSelfAttention(d_model, num_heads)

    torch.manual_seed(42)
    pytorch_mha = nn.MultiheadAttention(d_model, num_heads, batch_first=True)

    x = torch.randn(batch, seq, d_model)

    our_output, _ = our_mha(x)
    pytorch_output, _ = pytorch_mha(x, x, x)

    # 가중치가 다르므로 출력은 다를 수 있지만, shape은 동일해야 함
    assert our_output.shape == pytorch_output.shape, "Shape 불일치!"
    print("✅ PyTorch 구현과 shape 동일!")
    print(f"   Our output: {our_output.shape}")
    print(f"   PyTorch output: {pytorch_output.shape}")


test_vs_pytorch()

---

## Post-Quiz

### Q1. num_heads=8, d_model=512일 때 각 head의 d_k는?

```
당신의 답:
```

### Q2. head를 늘리면 계산량이 증가하나요?

```
당신의 답:
```

### Q3. Multi-Head의 장점을 한 문장으로?

```
당신의 답:
```


---
## 정답

<details>
<summary>👉 클릭하여 정답 보기</summary>

### Pre-Quiz

**Q1. Multi-Head 사용 이유**
> - 여러 관계 패턴을 **병렬로** 학습
> - head 1: 주어-동사, head 2: 수식어 관계 등
> - Single head로는 한 가지 패턴만 학습

**Q2. d_k 계산**
> d_k = d_model / num_heads = 512 / 8 = **64**

**Q3. 각 head가 학습하는 것**
> - 서로 다른 subspace에서의 attention 패턴
> - 문법적 관계, 의미적 관계, 위치 관계 등
> - 학습 후 각 head가 specialization됨

---

### Post-Quiz

**Q1. d_k**

> 512 / 8 = **64**

**Q2. 계산량**

> **증가하지 않음!** 총 d_model은 그대로이고 나눠서 계산하므로계산량은 동일. 오히려 병렬화 가능하여 효율적.

**Q3. 장점 한 문장**

> "여러 관계 패턴을 병렬로 학습하여 다양한 언어 구조를 동시에 포착"

</details>


---

## 구현 정답 코드

<details>
<summary>👉 클릭하여 정답 코드 보기</summary>

```python
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, num_heads: int, dropout: float = 0.0):
        super().__init__()
        assert d_model % num_heads == 0

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        seq_q = query.size(1)
        seq_k = key.size(1)

        # 1. Linear projection
        Q = self.W_q(query)  # (batch, seq_q, d_model)
        K = self.W_k(key)    # (batch, seq_k, d_model)
        V = self.W_v(value)  # (batch, seq_k, d_model)

        # 2. Reshape to (batch, seq, num_heads, d_k)
        Q = Q.view(batch_size, seq_q, self.num_heads, self.d_k)
        K = K.view(batch_size, seq_k, self.num_heads, self.d_k)
        V = V.view(batch_size, seq_k, self.num_heads, self.d_k)

        # 3. Transpose to (batch, num_heads, seq, d_k)
        Q = Q.transpose(1, 2)  # (batch, num_heads, seq_q, d_k)
        K = K.transpose(1, 2)  # (batch, num_heads, seq_k, d_k)
        V = V.transpose(1, 2)  # (batch, num_heads, seq_k, d_k)

        # 4. Reshape for attention: (batch*num_heads, seq, d_k)
        Q = Q.contiguous().view(batch_size * self.num_heads, seq_q, self.d_k)
        K = K.contiguous().view(batch_size * self.num_heads, seq_k, self.d_k)
        V = V.contiguous().view(batch_size * self.num_heads, seq_k, self.d_k)

        # 5. Attention (mask도 reshape 필요)
        if mask is not None:
            mask = mask.unsqueeze(1).expand(-1, self.num_heads, -1, -1)
            mask = mask.contiguous().view(batch_size * self.num_heads, seq_q, seq_k)

        output, weights = scaled_dot_product_attention(Q, K, V, mask)

        # 6. Reshape: (batch*num_heads, seq, d_k) -> (batch, num_heads, seq, d_k)
        output = output.view(batch_size, self.num_heads, seq_q, self.d_k)
        weights = weights.view(batch_size, self.num_heads, seq_q, seq_k)

        # 7. Transpose + concat: (batch, seq, d_model)
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_q, self.d_model)

        # 8. Output projection
        output = self.W_o(output)
        output = self.dropout(output)

        return output, weights
```

</details>
